<a href="https://colab.research.google.com/github/heidingaway/datapeople/blob/main/opengov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
from ckanapi import RemoteCKAN
import pandas as pd
import os
import zipfile
import yaml

In [52]:
TOKEN = "cac6fd9f-594a-4bcd-bf17-10295812d4c5"
rc = RemoteCKAN('https://open.canada.ca/data/en/', apikey=TOKEN)
result = rc.action.datastore_search(
    resource_id="bdb33e8c-53ef-4bae-9493-35f343191c02",
)

# Convert the 'records' to a pandas DataFrame
df = pd.DataFrame(result['records'])

# Add a new 'title' column using the content of 'GC_NM_AB_EN'
df['title'] = df['GC_NM_AB_EN']

# Remove special characters from the 'title' column
df['title'] = df['title'].str.replace(r'[^\w\s-]', '', regex=True)
df['title'] = df['title'].str.replace(r'[\s]+', '_', regex=True)

# Clean up 'id' column
df = df.rename(columns={'_id': 'id'})

print(df.head(2))

df.to_csv('output.csv', index=False)
print(f"DataFrame saved to output.csv")

   id GC_HIST_ID  GC_ID                                  GC_NM_OFF_EN  \
0   1    1000110  10001                          the Republic of Fiji   
1   2    1000210  10002  South Georgia and the South Sandwich Islands   

                                 GC_NM_OFF_FR  \
0                     la République des Fidji   
1  Géorgie du Sud-et-les Îles Sandwich du Sud   

                                    GC_NM_AB_EN  \
0                                          Fiji   
1  South Georgia and the South Sandwich Islands   

                                  GC_NM_AB_FR STAT_CD STAT_DESC_EN  \
0                                       Fidji       1       ACTIVE   
1  Géorgie du Sud-et-les Îles Sandwich du Sud       1       ACTIVE   

  STAT_DESC_FR  ...            EFF_FN_DT OBSERVATIONS_EN OBSERVATIONS_FR  \
0       ACTIVE  ...  2100-01-01T00:00:00            NULL            NULL   
1       ACTIVE  ...  2100-01-01T00:00:00            NULL            NULL   

  GC_PAR_ID GC_JUR_ID ISO_ALPHA_2_CD I

In [54]:
# Create a directory to store the files
output_dir = 'countryfiles'
os.makedirs(output_dir, exist_ok=True)

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Create the file name (e.g., row_0.md, row_1.md, etc.)
    file_name = os.path.join(output_dir, f"{row['title']}.md")

    # Convert the row to a dictionary
    row_dict = row.to_dict()

    # Convert the dictionary to YAML format
    yaml_content = yaml.dump(row_dict, default_flow_style=False)

    # Write the YAML content to the file (with .md extension)
    with open(file_name, 'w') as f:
        f.write(yaml_content)

print(f"Files with YAML content (saved as .md) are in the '{output_dir}' directory.")

# Create a zip file containing the Markdown files
zip_file_name = 'countryfiles.zip'
with zipfile.ZipFile(zip_file_name, 'w') as zipf:
    for root, _, files in os.walk(output_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, output_dir))

print(f"Markdown files zipped to '{zip_file_name}'.")

Files with YAML content (saved as .md) are in the 'countryfiles1' directory.
Markdown files zipped to 'countryfiles1.zip'.
